Este notebook utiliza o **FastText de 50 dimensões** na base de dados do BTG. 

São aplicados os modelos **'Naive Bayes'** (que retorna uma acurácia de 50%) e a **'Regressão Logística'** (que retorna uma acurácia de 72%).

# Setup

In [ ]:
pip install gensim==4.3.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
import pandas as pd
import numpy as np
from gensim.models import KeyedVectors
import numpy as np
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
from nltk.stem import WordNetLemmatizer

In [10]:
# Download dos recursos necessários do NLTK
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [30]:
df_2 = pd.read_csv('/content/drive/MyDrive/base/lematizacao.csv')

In [31]:
df_2 = df_2.rename(columns={'Unnamed: 0': 'id'})
df_2

,id,autor,texto,sentimento,tokens
0,0,v8_capital,Confira os resultados dos nossos fundos no mês...,NEUTRAL,"['confira', 'o', 'resultado', 'de o', 'nosso',..."
1,1,winthegame_of,A Alvarez & Marsal estará conosco no Sportainm...,NEUTRAL,"['Alvarez', 'Marsal', 'estara', 'conosco', 'sp..."
2,2,marta_bego,#Repost btgpactual with make_repost ・・・ Entend...,NEUTRAL,"['repost', 'btg', 'With', 'makerepost', 'enten..."
3,3,lmviapiana,Minuto touro de ouro,POSITIVE,"['minuto', 'touro', 'ouro']"
4,4,vanilson_dos,@ricktolledo Sim,NEUTRAL,"['Ricktolledo', 'sim']"
...,...,...,...,...,...
12188,12350,onixcapital,Uma noite de encontros e muito conhecimento on...,NEUTRAL,"['um', 'noite', 'encontro', 'muito', 'conhecim..."
12189,12351,amgcapitalinvest,Erros financeiros para eliminar antes dos 30 a...,NEUTRAL,"['erro', 'financeiro', 'eliminar', 'antes', 'd..."
12190,12352,jayipacderota,Estou muito grato por todo o esforço que você ...,POSITIVE,"['estar', 'muito', 'grato', 'todo', 'esforco',..."
12191,12353,moinho_cultural,"""👉GUADAKAN está chegando!!!! A aula pública e...",NEUTRAL,['dorsodamaocomdedoindicadorapontandoparaadire...


In [32]:
path = '/content/drive/MyDrive/cbow_s50.txt'

In [14]:
model = KeyedVectors.load_word2vec_format(path)

# Pré processamento

In [ ]:
# Acrescenta a coluna de sentimento
df_2['sentimento'] = df_2['sentimento'].map({'NEUTRAL': 0, 'POSITIVE': 1, 'NEGATIVE': -1})

## Definição de funções

In [33]:
# Função para vetorizar um token
def vetorizar_token(token):
    vetor = np.zeros(model.vector_size) # Inicializa vetor de zeros com a mesma dimensão
    if token in model: # Verifica se a palavra está no word2vec treinado
        vetor = model[token] # Adiciona o valor do vetor
    return vetor

,id,autor,texto,sentimento,tokens,tokens_preprocessados
0,0,v8_capital,Confira os resultados dos nossos fundos no mês...,0,"['confira', 'o', 'resultado', 'de o', 'nosso',...","[confira, resultados, fundos, mês, fevereiro/2..."
1,1,winthegame_of,A Alvarez & Marsal estará conosco no Sportainm...,0,"['Alvarez', 'Marsal', 'estara', 'conosco', 'sp...","[alvarez, marsal, estará, conosco, sportainmet..."
2,2,marta_bego,#Repost btgpactual with make_repost ・・・ Entend...,0,"['repost', 'btg', 'With', 'makerepost', 'enten...","[repost, btgpactual, with, make_repost, ・・・, e..."
3,3,lmviapiana,Minuto touro de ouro,1,"['minuto', 'touro', 'ouro']","[minuto, touro, ouro]"
4,4,vanilson_dos,@ricktolledo Sim,0,"['Ricktolledo', 'sim']","[ricktolledo, sim]"
...,...,...,...,...,...,...
12188,12350,onixcapital,Uma noite de encontros e muito conhecimento on...,0,"['um', 'noite', 'encontro', 'muito', 'conhecim...","[noite, encontros, conhecimento, onde, cliente..."
12189,12351,amgcapitalinvest,Erros financeiros para eliminar antes dos 30 a...,0,"['erro', 'financeiro', 'eliminar', 'antes', 'd...","[erros, financeiros, eliminar, antes, 30, anos..."
12190,12352,jayipacderota,Estou muito grato por todo o esforço que você ...,1,"['estar', 'muito', 'grato', 'todo', 'esforco',...","[grato, todo, esforço, rendeu, investimento, r..."
12191,12353,moinho_cultural,"""👉GUADAKAN está chegando!!!! A aula pública e...",0,['dorsodamaocomdedoindicadorapontandoparaadire...,"[``, 👉guadakan, chegando, aula, pública, forma..."


In [ ]:
# Função para pré-processar uma frase
def preprocessar_frase(frase):
    # Tokenização
    tokens = word_tokenize(frase)
    
    # Remoção de stop words
    stop_words = set(stopwords.words('portuguese'))
    tokens = [token for token in tokens if token.lower() not in stop_words]
    
    # Remoção de pontuação e caracteres especiais
    tokens = [token for token in tokens if token not in string.punctuation]
    
    # Normalização de texto
    tokens = [token.lower() for token in tokens]
    
    return tokens

## Teste e aplicação

In [38]:
teste = ['Estou muito grato por todo o esforço', 'estou bem feliz, para falar a verdade']
result = []
for item in teste:
    result.append(preprocessar_frase(item))

print(result)

[['grato', 'todo', 'esforço'], ['bem', 'feliz', 'falar', 'verdade']]


In [ ]:
# Aplicar a função 'preprocessar_frase' a todas as frases
df_2['tokens_preprocessados'] = df_2['texto'].apply(preprocessar_frase)
df_2

# NAIVE BAYES

In [16]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

In [34]:
y = df_2['sentimento']

# Convert the tokenized sentences into text
sentences = [' '.join(tokens) for tokens in df_2['tokens_preprocessados']]

# Convert text data to numerical features using TF-IDF
vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(sentences)  # Assuming 'tokens' contains preprocessed tokens

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X_tfidf, y, test_size=0.2)

# Naive Bayes
classifier = GaussianNB()

# Training the classifier
classifier.fit(X_train.toarray(), y_train)  # Convert X_train to array

# Prediction on the test set
y_pred = classifier.predict(X_test.toarray())  # Convert X_test to array

# Validating the model
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

# Print classification report
report = classification_report(y_test, y_pred)
print("Classification Report:")
print(report)

Accuracy: 0.4797047970479705
Classification Report:
              precision    recall  f1-score   support

          -1       0.30      0.66      0.42       516
           0       0.63      0.29      0.40      1056
           1       0.63      0.60      0.61       867

    accuracy                           0.48      2439
   macro avg       0.52      0.52      0.48      2439
weighted avg       0.56      0.48      0.48      2439



# REGRESSÃO LOGÍSTICA

In [35]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Convert the tokenized sentences into text
sentences = [' '.join(tokens) for tokens in df_2['tokens_preprocessados']]

# Define the target variable 'y' from your dataset
y = df_2['sentimento']

# Apply CountVectorizer to the sentences
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(sentences)

# Normalize the vectors
scaler = StandardScaler()
X = scaler.fit_transform(X.toarray())

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the logistic regression model
logreg = LogisticRegression()

# Train the model
logreg.fit(X_train, y_train)

# Make predictions on the test data
y_pred = logreg.predict(X_test)

# Calculate the accuracy of the predictions
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))



/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


Accuracy: 72.20%
